# **공연별 예매된 좌석, 예매되지 않은 좌석 테이블 구조 만들기**

In [1]:
import pandas as pd 
import sys
import numpy as np
import time
import warnings

warnings.filterwarnings(action = 'ignore')

In [2]:
seat_information = pd.read_csv("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/seat_information_angle_ver3.csv",encoding = 'utf-8')
df = pd.read_parquet("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/dataframe_reduced_size.parquet")

In [3]:
seat_information.head(3)

,Unnamed: 0,층,블록,열,넘버,전체_좌석,X,Y,Z,대칭점,무대까지의 거리,좌우면적시야각,상하면적시야각,시야방해점수,좌석공간넓음,휠체어석,1/거리**3,무대정면과의각도,좌우시선각도,상하시선각도
0,0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9,1551.713247,15.219900,33.948421,100,1,0,2.676490e-10,0.350,55.518253,3.436010
1,1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8,1514.433888,16.945594,34.331746,100,1,0,2.879049e-10,0.367,54.459722,3.520697
2,2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7,1477.698887,18.869853,34.695385,100,1,0,3.099147e-10,0.385,53.347959,3.608334


In [4]:
# merge 하기 위해 전체_좌석 컬럼을 seat 라는 컬럼 명으로 변경시커주자 

seat_information = seat_information.rename(columns = {'전체_좌석':'seat'})

display(seat_information.head())

,Unnamed: 0,층,블록,열,넘버,seat,X,Y,Z,대칭점,무대까지의 거리,좌우면적시야각,상하면적시야각,시야방해점수,좌석공간넓음,휠체어석,1/거리**3,무대정면과의각도,좌우시선각도,상하시선각도
0,0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9,1551.713247,15.219900,33.948421,100,1,0,2.676490e-10,0.350,55.518253,3.436010
1,1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8,1514.433888,16.945594,34.331746,100,1,0,2.879049e-10,0.367,54.459722,3.520697
2,2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7,1477.698887,18.869853,34.695385,100,1,0,3.099147e-10,0.385,53.347959,3.608334
3,3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1층 E블록1열 6,1440.234009,21.009736,35.069575,100,1,0,3.347348e-10,0.404,52.200118,3.702327
4,4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1층 E블록1열 5,1403.862529,23.314833,35.426958,100,1,0,3.614317e-10,0.422,51.045580,3.798387


In [5]:
print('좌석 정보 데이터 프레임 : ', seat_information.shape)
print('메모리 정리 된 데이터 프레임 :', df.shape)

좌석 정보 데이터 프레임 :  (2505, 20)
메모리 정리 된 데이터 프레임 : (236966, 33)


# **해야 할 일**

각 공연 일, performance code 별 예매 된 좌석에 대해서는 예매 여부를 담은 테이블을 만들어야 함

In [6]:
print('유니크한 공연일 개수 : ',df['전체공연시간'].nunique())

유니크한 공연일 개수 :  162


# **빅 콘테스트 의문사항**

In [7]:
unique_combinations = df.groupby(['genre','performance_code', '전체공연시간']).size().reset_index(name='count')

cond = unique_combinations['count'] > 2505

unique_combinations[cond]

,genre,performance_code,전체공연시간,count
8107,클래식,902,2019-03-14 20:00:00,2580
8922,클래식,916,2019-05-19 17:00:00,2961
10526,클래식,989,2023-05-21 17:00:00,2718
11483,클래식,1182,2022-12-20 19:30:00,2558
16756,클래식,1725,2021-03-31 19:30:00,3030
16869,클래식,1828,2019-07-13 17:00:00,2723
17027,클래식,1869,2019-07-02 20:00:00,2574
18919,클래식,2158,2022-10-20 20:00:00,2743


전체 공연 좌석이 2505 개인데 2505개보다 많은 좌석이 판매된 경우가 있음

살펴보니 같은 좌석에서 다른 사람이 다른 시간에 예매를 한 것으로 나옴

이렇게 되면 해당 공연의 예매율은 100% 를 넘어갈텐데

우선 그룹핑 할때는 이런 일이 없도록 drop_duplicate 를 시행해주자

# **좌석 좌표와 병합 후 없는 좌석은 예매 여부 0으로 하기**

# **병합 전 seat_information.csv 파일 수정**

In [8]:
cond = seat_information['블록'].str.contains('BOX')

# Box석의 경우 BOX 의 번호가 열이 되도록 수정

seat_information.loc[cond, '열'] = seat_information.loc[cond, '블록'].map(lambda x: x[3:])
seat_information.loc[cond, '블록'] = seat_information.loc[cond, '블록'].map(lambda x: x[:3])

In [9]:
print('맨 위 5개의 행')
display(seat_information.head())

print('맨 아래 5개의 행')
display(seat_information.tail())

맨 위 5개의 행


,Unnamed: 0,층,블록,열,넘버,seat,X,Y,Z,대칭점,무대까지의 거리,좌우면적시야각,상하면적시야각,시야방해점수,좌석공간넓음,휠체어석,1/거리**3,무대정면과의각도,좌우시선각도,상하시선각도
0,0,1층,A블록,1,1,1층 A블록1열 1,1451,542,-93,1층 E블록1열 9,1551.713247,15.219900,33.948421,100,1,0,2.676490e-10,0.350,55.518253,3.436010
1,1,1층,A블록,1,2,1층 A블록1열 2,1406,555,-93,1층 E블록1열 8,1514.433888,16.945594,34.331746,100,1,0,2.879049e-10,0.367,54.459722,3.520697
2,2,1층,A블록,1,3,1층 A블록1열 3,1361,568,-93,1층 E블록1열 7,1477.698887,18.869853,34.695385,100,1,0,3.099147e-10,0.385,53.347959,3.608334
3,3,1층,A블록,1,4,1층 A블록1열 4,1315,580,-93,1층 E블록1열 6,1440.234009,21.009736,35.069575,100,1,0,3.347348e-10,0.404,52.200118,3.702327
4,4,1층,A블록,1,5,1층 A블록1열 5,1270,591,-93,1층 E블록1열 5,1403.862529,23.314833,35.426958,100,1,0,3.614317e-10,0.422,51.045580,3.798387


맨 아래 5개의 행


,Unnamed: 0,층,블록,열,넘버,seat,X,Y,Z,대칭점,무대까지의 거리,좌우면적시야각,상하면적시야각,시야방해점수,좌석공간넓음,휠체어석,1/거리**3,무대정면과의각도,좌우시선각도,상하시선각도
2500,2500,2층,BOX,6,2,2층 BOX6 2,-1432,505,383,2층 BOX1 1,1565.994253,21.173142,34.169657,93,0,0,2.603931e-10,0.333,21.530131,14.156604
2501,2501,2층,BOX,6,3,2층 BOX6 3,-1500,565,428,2층 BOX1 4,1659.038577,21.144470,32.190452,96,0,0,2.189932e-10,0.352,20.315795,14.950280
2502,2502,2층,BOX,6,4,2층 BOX6 4,-1454,617,428,2층 BOX1 3,1636.456232,23.354990,31.913243,92,0,0,2.281849e-10,0.391,17.961738,15.161505
2503,2503,2층,BOX,6,5,2층 BOX6 5,-1522,677,483,2층 BOX1 6,1734.388077,23.078793,30.097223,89,0,0,1.916731e-10,0.406,16.975564,16.169769
2504,2504,2층,BOX,6,6,2층 BOX6 6,-1568,624,483,2층 BOX1 5,1755.360077,21.128253,30.390770,90,0,0,1.848849e-10,0.370,19.254978,15.971384


In [10]:
class seat_merging:
    
    def __init__(self,data,seat):
        '''
        예매 된 공연 테이블과 좌석 정보를 담은 테이블을 가져오기
        '''
        self.df = data 
        self.seat = seat
        
        self.seat['예매여부'] = 1 # merging 후 특정 조건에 따라 예매 여부를 0으로 mapping 하여 수정 할 예정 
        
        
        self.genres_df_dict = {}
        self.genres_merged_dict = {}
        self.genres_num_count = {}

            
    def merging_seat_information(self):
        
        # 병합 후 채워야 하는 컬럼들 
        
        arr = ['play_date','play_st_time','ticket_cancel','performance_code',
                    'pre_open_date','open_date','genre','place','running_time','intermission',
                    '공연연도','공연월','공연일','공연연월','전체공연시간']
                
        unique_performance = sorted(self.df['전체공연시간'].unique())        

        merging_df_list = []
        
        for uni in unique_performance: # 장르 별, 공연 별로 merge 시키기
            
            cond = self.df['전체공연시간'] == uni
            
            cond_df = self.df[cond]
            
            cancle_cond = cond_df['ticket_cancel'] == 1 # 티켓을 취소한 경우도 제거해야 한다 
            
            cond_df = cond_df[~cancle_cond]
            
            # 좌석의 중복값을 제거해야함 
            # 전체 거래시간 별로 오름차순 정렬 후 마지막에 중복값중 마지막 중복값 제외 drop 하기
            # 마지막 거래가 실제 거래라고 생각하기로 함
            
            cond_df = cond_df.sort_values(by = '전체거래시간')
            cond_df = cond_df.drop_duplicates(subset = 'seat', keep = 'last')
            
            # seat information 과 유니크한 공연 별 데이터 프레임을 병합 
            # 이렇게 하게 되면 merging_df 는 2505 행의 데이터프레임이 완성 될 것임 
            merging_df = pd.merge(self.seat[['seat','층', '블록', '열', '넘버','무대까지의 거리', '좌우면적시야각', '상하면적시야각', '시야방해점수', '좌석공간넓음', '휠체어석', '1/거리**3',
                                                '무대정면과의각도', '좌우시선각도', '상하시선각도','예매여부']], cond_df, on = 'seat', how = 'left')

            not_purcahce_cond = merging_df['전체공연시간'].isna()
            
            merging_df.loc[not_purcahce_cond, '예매여부'] = 0 # 예매가 안됐던 경우에 대해서는 예매 여부를 0으로 변경하기 
            
            for fill_col in arr: 
                if merging_df[fill_col].notna().any():
                    original_value = merging_df.loc[merging_df[fill_col].notna(), fill_col].iloc[0]
                    merging_df[fill_col] = merging_df[fill_col].fillna(original_value)
                
            merging_df_list.append(merging_df) # merging_df_list 에 차곡 차곡 2505 석의 공연 별로 채워주자 
                    
        result = pd.concat(merging_df_list)
        
        # 장르하고 place 는 drop 하자 이미 클래식과 콘서트홀만 연동된다
        
        result = result.drop(columns = ['genre', 'place'], axis = 1)
        
        return result

In [11]:
merging_method = seat_merging(df.drop(['층','석','세부좌석'],axis = 1),
                                seat_information) # 기존 df 의 층,석,세부좌석이 seat_information 과 같은 내용이니 하나는 drop 하자

In [12]:
merging_df = merging_method.merging_seat_information()

In [13]:
print('예매 여부가 1인 경우')
display(merging_df[merging_df['예매여부'] == 1].sample(5))

print('예매 여부가 0인 경우')
display(merging_df[merging_df['예매여부'] == 0].sample(5))

예매 여부가 1인 경우


,seat,층,블록,열,넘버,무대까지의 거리,좌우면적시야각,상하면적시야각,시야방해점수,좌석공간넓음,...,open_date,running_time,intermission,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간
201,1층 A블록5열 2,1층,A블록,5,2,1801.714739,22.460544,26.899532,100,0,...,2022-12-03,120.0,0.0,N,2022.0,12.0,2022-12-10,2022-12-01,2022-12-10 17:00:00,2022-11-03 11:44:00
1171,1층 D블록21열 9,1층,D블록,21,9,2809.091846,29.183935,7.653209,100,0,...,2019-10-14,120.0,15.0,N,2019.0,11.0,2019-11-24,2019-11-01,2019-11-24 17:00:00,2019-10-25 11:19:00
1763,2층 E블록4열 4,2층,E블록,4,4,2785.340195,28.016594,10.763053,99,0,...,2018-10-29,140.0,20.0,N,2018.0,12.0,2018-12-08,2018-12-01,2018-12-08 17:00:00,2018-11-23 13:24:00
490,1층 B블록10열 4,1층,B블록,10,4,1789.552458,43.167276,15.697251,100,0,...,2019-01-27,120.0,20.0,Y,2019.0,5.0,2019-05-19,2019-05-01,2019-05-19 17:00:00,2019-01-27 10:03:00
2409,3층 B블록7열 5,3층,B블록,7,5,3461.767468,23.119625,9.503320,97,0,...,2019-06-29,100.0,15.0,Y,2019.0,11.0,2019-11-20,2019-11-01,2019-11-20 20:00:00,2019-06-29 16:09:00


예매 여부가 0인 경우


,seat,층,블록,열,넘버,무대까지의 거리,좌우면적시야각,상하면적시야각,시야방해점수,좌석공간넓음,...,open_date,running_time,intermission,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간
1834,2층 D블록5열 10,2층,D블록,5,10,2940.874190,28.343237,8.015358,98,0,...,2022-04-23,120.0,15.0,NaN,2022.0,6.0,2022-06-09,2022-06-01,2022-06-09 19:30:00,NaT
2173,3층 A블록3열 2,3층,A블록,3,2,2800.180887,23.791143,15.829678,79,0,...,2022-06-12,120.0,15.0,NaN,2022.0,7.0,2022-07-16,2022-07-01,2022-07-16 17:00:00,NaT
839,1층 B블록16열 10,1층,B블록,16,10,2287.012024,36.887668,8.010371,100,0,...,2022-06-26,100.0,15.0,NaN,2022.0,12.0,2022-12-31,2022-12-01,2022-12-31 17:00:00,NaT
1291,합창석 H블록1열 14,합창석,H블록,1,14,1102.702589,44.179677,55.113583,91,0,...,2019-05-31,110.0,20.0,NaN,2019.0,7.0,2019-07-19,2019-07-01,2019-07-19 20:00:00,NaT
2111,3층 M블록1열 10,3층,M블록,1,10,3764.865602,22.810851,6.900351,99,0,...,2022-08-22,100.0,15.0,NaN,2022.0,11.0,2022-11-27,2022-11-01,2022-11-27 17:00:00,NaT


# **잘 완료 되었고 깃허브에서 연동 할 수 있도록 각 데이터 프레임의 용량을 낮춘 후 저장하자**

In [14]:
file_path = 'C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/'

merging_df.to_parquet(file_path + '빈좌석_포함_클래식_데이터.parquet',index = False)